## 1. Install and Import Dependencies

In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/4.9 MB ? eta -:--:--
     --- ------------------------------------ 0.4/4.9 MB 11.5 MB/s eta 0:00:01
     --------------- ------------------------ 2.0/4.9 MB 20.8 MB/s eta 0:00:01
     ------------------------ --------------- 3.0/4.9 MB 23.8 MB/s eta 0:00:01
     ----------------------------- ---------- 3.6/4.9 MB 21.0 MB/s eta 0:00:01
     ----------------------------- ---------- 3.7/4.9 MB 16.7 MB/s eta 0:00:01
     ------------------------------------ --- 4.5/4.9 MB 16.0 MB/s eta 0:00:01
     ---------------------------------------  4.9/4.9 MB 16.5 MB/s eta 0:00:01
     ---------------------------------------  4.9/4.9 MB 16.5 MB/s eta 0:00:01
     ---------------------------------------  4.9/4.9 MB 16.5 MB/s eta 0:00:01
     ---------------------------------------  4.9/4.9 MB 16.5 MB/s eta 0:00:01
  

     ---- ----------------------------------- 0.3/2.7 GB 14.6 MB/s eta 0:02:45
     ---- ----------------------------------- 0.3/2.7 GB 12.6 MB/s eta 0:03:11
     ---- ----------------------------------- 0.3/2.7 GB 12.6 MB/s eta 0:03:11
     ---- ----------------------------------- 0.3/2.7 GB 12.1 MB/s eta 0:03:18
     ---- ----------------------------------- 0.3/2.7 GB 12.4 MB/s eta 0:03:14
     ---- ----------------------------------- 0.3/2.7 GB 12.8 MB/s eta 0:03:07
     ---- ----------------------------------- 0.3/2.7 GB 13.4 MB/s eta 0:02:59
     ---- ----------------------------------- 0.3/2.7 GB 12.8 MB/s eta 0:03:07
     ---- ----------------------------------- 0.3/2.7 GB 12.8 MB/s eta 0:03:07
     ---- ----------------------------------- 0.3/2.7 GB 11.9 MB/s eta 0:03:21
     ---- ----------------------------------- 0.3/2.7 GB 11.3 MB/s eta 0:03:32
     ---- ----------------------------------- 0.3/2.7 GB 10.4 MB/s eta 0:03:50
     ---- ----------------------------------- 0.3/2.

In [4]:
!pip install transformers requests beautifulsoup4 pandas numpy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

## 2. Instantiate Model

In [6]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

C:\Users\georg\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\georg\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

## 3. Encode and Calculate Sentiment

In [30]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors = 'pt')

In [31]:
tokens

tensor([[  101, 10197, 10140, 12050, 10502, 12296, 10598, 10662, 16197,   119,
         11838,   102]])

In [32]:
tokenizer.decode(tokens[0])

'[CLS] it was good but couldve been better. great [SEP]'

In [33]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [34]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [35]:
int(torch.argmax(result.logits)) + 1

4

## 4. Collect Reviews

In [44]:
r  = requests.get("https://www.yelp.com/biz/mejico-sydney-2")
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class': regex})
reviews = [result.text for result in results]
reviews

["Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future.",
 'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to che

## 5. Load Reviews into DataFrame and Score

In [48]:
df = pd.DataFrame(np.array(reviews), columns=['reviews'])

In [51]:
df.head()

,reviews
0,Seated without a booking on a super busy Satur...
1,The food was decent not great.. We had the gu...
2,"Food was okay, guacamole was below average. Se..."
3,The food and service here was really good. It...
4,Visiting from Texas and decided to give this r...


In [59]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors = 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits)) + 1

df['sentiment'] = df['reviews'].apply(lambda x: sentiment_score(x[:512]))

In [61]:
df

,reviews,sentiment
0,Seated without a booking on a super busy Satur...,5
1,The food was decent not great.. We had the gu...,2
2,"Food was okay, guacamole was below average. Se...",2
3,The food and service here was really good. It...,5
4,Visiting from Texas and decided to give this r...,5
5,Don't come here expecting legit Mexican food b...,3
6,Out of all the restaurants that I tried in Syd...,5
7,"Great atmosphere, attentive service, solid mar...",3
8,We came here on a Thursday night @ 5pm and by ...,4
9,The food is fresh and tasty. The scallop cevi...,4
